In [1]:
from dataclasses import asdict

import polars as pl
import os
from starkware.cairo.lang.cairo_constants import DEFAULT_PRIME
from starkware.cairo.lang.compiler.cairo_compile import compile_cairo_files
from starkware.cairo.lang.vm.cairo_runner import CairoRunner

In [2]:
# %% Initialize runner from a .cairo program
PATH_TO_MAIN = "../cairo0/fibonacci"
MAIN_FILENAME = "main.cairo"

runner = CairoRunner(
    program = compile_cairo_files(
        files = [os.path.join(PATH_TO_MAIN,MAIN_FILENAME)],
        prime = DEFAULT_PRIME,
        cairo_path = [PATH_TO_MAIN]
        ),
    layout = "plain"
)

runner.initialize_segments()
runner.initialize_main_entrypoint()
runner.initialize_vm({})

runner.run_until_pc(runner.final_pc)
runner.end_run()

In [3]:
# %% Get memory and trace from runner after relocation
runner.relocate()

memory = pl.DataFrame(
    {
        "address": runner.relocated_memory.data.keys(),
        "value": [f"{v:016x}" for v in runner.relocated_memory.data.values()],
    }
)
trace = pl.DataFrame([asdict(x) for x in runner.relocated_trace])

In [4]:
# %% Instructions table
instruction_table = (
    pl.concat([trace["pc"], trace["pc"] + 1])
    .unique()
    .to_frame()
    .join(memory, left_on="pc", right_on="address", how="left")
    .filter(pl.col("value").str.len_chars() == 16)
    .with_columns(
        instruction=pl.col("value").str.to_integer(base=16).fill_null(0),
        off_dst=pl.col("value").str.slice(12, 4).str.to_integer(base=16).fill_null(0),
        off_op0=pl.col("value").str.slice(8, 4).str.to_integer(base=16).fill_null(0),
        off_op1=pl.col("value").str.slice(4, 4).str.to_integer(base=16).fill_null(0),
        flags=(
            pl.col("value")
            .str.slice(0, 4)
            .str.to_integer(base=16)
            .fill_null(0)
            .map_elements(lambda x: f"{x:016b}")
            .map_elements(lambda x: [int(x[:i], 2) for i in range(1, 17)])
        ),
    )
    .with_columns(flags=pl.col("flags").list.to_struct(fields=lambda i: f"f_{15 - i}"))
    .drop("value")
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/tmp/ipykernel_34711/1148067401.py:22: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  .with_columns(flags=pl.col("flags").list.to_struct(fields=lambda i: f"f_{15 - i}"))


In [5]:
# %% Decoding assertion
constraint = pl.col("off_dst") + 2**16 * pl.col("off_op0") + 2**32 * pl.col("off_op1") + 2**48 * pl.col("f_0") == pl.col("instruction")
assert instruction_table.unnest("flags").select(valid=constraint)["valid"].all()

In [6]:
# %% Flags assertion
diff = (-2 * pl.col("value") + pl.col("value").shift(-1)).fill_null(0)
last_flag = pl.Series(range(len(instruction_table) * 16)) % 16
constraint = diff * (diff - 1) * ~(last_flag == 15) == 0

assert (
    instruction_table["flags"]
    .struct.unnest()
    .transpose()
    .unpivot()
    .select(valid=constraint)["valid"]
    .all()
)